In [2]:
from tqdm.notebook import tqdm
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
import re
import numpy as np
import ast

In [34]:
df = pd.read_pickle("../data/events/df_events 2025-01-10_16-41-46.pkl")

In [35]:
columns_order = ['StartDate', 'StartTime',  "EndDate", "EndTime", 'StartDateTime', "EndDateTime"]
for col in df.columns:
    if col not in columns_order:
        columns_order.append(col)


In [36]:
# reorder columns 
df = df[columns_order]  

In [37]:
df = df.sample(frac=1, random_state=42).reset_index(drop=True)



In [ ]:
df.City.value_counts()


In [41]:
df.City = df.City.str.replace("im Breisgau", "")
df.City = df.City.str.replace("im breisgau", "")
df.City = df.City.str.strip()

In [42]:
df.rename(columns = {
    "StartTimeIsAssumed": "StartTimeIsUnknown",
}, inplace=True)

In [48]:
df.Street = df.Street.str.replace("Dresden, Germany", "")
df.Street = df.Street.str.replace("Dresden, Alemania", "")

In [ ]:
flt =( df.lat.isna()) & (~df.Street.isna())
print(len(df))
print(len(df[flt]))
df[flt]

In [ ]:
df[df.Street.str.contains("Theaterplatz", na=False)]

In [ ]:
df.Source.value_counts()


In [ ]:
df.Street = df.Street.str.lower()

In [55]:
df.Street = df.Street.str.replace("str.", "straße")

In [57]:
df.Street = df.Street.str.replace(",", "")

In [62]:
df.Street = df.Street.str.replace("\d{5}", "", regex=True)
df.HouseNumber = df.HouseNumber.str.replace("\d{5}", "", regex=True)

In [ ]:
df.HouseNumber.unique()

In [ ]:
df.Street.unique()

In [ ]:
flt = df.lat.isna()
for i, row in df.loc[flt].iterrows():
    

In [ ]:
df.groupby(['City', 'Street', 'HouseNumber'])['latitude'].transform(lambda x: x.fillna(x.mean()))
